## Startup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

import os
import pickle

In [ ]:
random_state=42
np.random.seed(random_state)

In [ ]:
from scripts.params import get_params

params = get_params()

## Data Retrieval

In [ ]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

In [ ]:
name=f'finaldf_train_{params["tablename"]}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)


In [ ]:
df.head()

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,YPFD.BA_rets,...,BBAR.BA_gk_vol,BBAR_rets,BBAR_log_rets,BBAR_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
2013-01-03,0.007552,0.007524,0.000129,0.010616,0.010560,0.000677,-0.012748,-0.012830,0.001228,-0.006863,...,0.000169,-0.005725,-0.005742,0.000960,0.008830,0.008791,0.000014,0.001247,0.001246,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.006303,-0.006323,0.000208,-0.010043,-0.010094,0.000554,0.004936,...,0.000406,-0.019194,-0.019380,0.000635,0.018043,0.017882,0.000133,-0.005727,-0.005743,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,0.002114,0.002112,0.000063,-0.014493,-0.014599,0.000517,0.010805,...,0.000492,0.015655,0.015534,0.000511,-0.002488,-0.002492,0.000048,-0.009769,-0.009817,0.000022
2013-01-08,0.008285,0.008251,0.000082,-0.008439,-0.008474,0.000153,-0.016176,-0.016309,0.001085,0.049563,...,0.000438,-0.015414,-0.015534,0.000642,0.015356,0.015239,0.000064,-0.001117,-0.001118,0.000082
2013-01-09,0.017826,0.017669,0.000273,0.000000,0.000000,0.000000,0.011958,0.011887,0.005238,0.000000,...,0.000000,-0.003914,-0.003922,0.000147,-0.008671,-0.008709,0.001065,0.017245,0.017098,0.000273


In [ ]:
tickerlist=params["tickerlist"]

## HMM Training

In [ ]:
range_states=range(1,16)
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range_states)
emptydf.fillna(np.inf, inplace=True)
results_dict_df={stock:emptydf for stock in tickerlist}

In [ ]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

In [ ]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]

    param_dict={
        "covariance_type" : "diag", 
        "n_iter" : 500,
        "random_state" : random_state
        #no voy a usar startprob_prior por devlog 20-06-23
        }

    for nstate in range_states:
        model = hmm.GaussianHMM(n_components= nstate, **param_dict, verbose=False)
        results = model.fit(insample_data)

        convergence=results.monitor_.converged
        # esta es la condición de si el modelo convergió

        all_states_found=np.isclose(a=(model.transmat_.sum(axis=1)), b=1).all()
        # esta es la condición de que todos los estados (nstates) hayan sido observados
        # si no, alguna fila en la matriz de transición del modelo son 0.
        # el errormsg es "Some rows of transmat_ have zero sum because no transition from the state was ever observed".

        startprob_check = (model.startprob_.sum()==1)
        # esta es la condición de que los estados al inicializar estén definidos
        
        good_model = convergence and all_states_found and startprob_check

        if good_model:
            try:
                results_dict_df[stock].loc[nstate, "AIC"]=model.aic(insample_data)
                results_dict_df[stock].loc[nstate, "BIC"]=model.bic(insample_data)
            except ValueError:
                pass
        else: 
            print(">"*10,f"{stock} {nstate} did not converge")
            results_dict_df[stock].loc[nstate, "BIC"]=np.inf
            results_dict_df[stock].loc[nstate, "BIC"]=np.inf

Model is not converging.  Current: 18854.600714436052 is not greater than 18855.73073827267. Delta is -1.1300238366166013
Model is not converging.  Current: 18150.45060648472 is not greater than 18151.012747677036. Delta is -0.5621411923166306
Model is not converging.  Current: 18859.18589914981 is not greater than 18864.433488621307. Delta is -5.247589471498941


>>>>>>>>>> ^MERV 2 did not converge


Model is not converging.  Current: 18906.152865429125 is not greater than 18906.40679425361. Delta is -0.2539288244843192
Model is not converging.  Current: 18184.342352220236 is not greater than 18184.37392065687. Delta is -0.03156843663600739
Model is not converging.  Current: 18893.543841853665 is not greater than 18897.57606814377. Delta is -4.032226290106337
Model is not converging.  Current: 18904.941147439862 is not greater than 18908.745436699235. Delta is -3.8042892593730357
Model is not converging.  Current: 18871.28534979241 is not greater than 18876.603915613836. Delta is -5.318565821424272
Model is not converging.  Current: 18198.450041297587 is not greater than 18198.454556353892. Delta is -0.004515056305535836
c:\Users\alfredo.sampron\AppData\Local\miniconda3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
Model is not converging.  Current: 18900.05218193964 i

>>>>>>>>>> ^MERV 15 did not converge


Model is not converging.  Current: 17746.98884698709 is not greater than 17749.887170814392. Delta is -2.898323827303102
Model is not converging.  Current: 17767.738850783353 is not greater than 17768.03631132645. Delta is -0.2974605430972588
Model is not converging.  Current: 17747.379395552358 is not greater than 17748.469048154537. Delta is -1.0896526021788304
Model is not converging.  Current: 17148.048118381404 is not greater than 17149.571333818105. Delta is -1.5232154367004114
Model is not converging.  Current: 17741.233619404436 is not greater than 17743.302334055254. Delta is -2.068714650817128
Model is not converging.  Current: 17726.79941652626 is not greater than 17726.83641027017. Delta is -0.03699374390998855
Model is not converging.  Current: 17753.78543565406 is not greater than 17754.58585606698. Delta is -0.8004204129210848
Model is not converging.  Current: 17176.803443578803 is not greater than 17176.81466860932. Delta is -0.011225030517380219
Model is not convergin

...

Model is not converging.  Current: 16293.223092897138 is not greater than 16297.273067091071. Delta is -4.04997419393294


>>>>>>>>>> BBAR 7 did not converge


Model is not converging.  Current: 17104.214221790317 is not greater than 17108.370578226477. Delta is -4.156356436160422
Model is not converging.  Current: 17093.564168934616 is not greater than 17093.57013615478. Delta is -0.005967220164166065
Model is not converging.  Current: 17070.867095785892 is not greater than 17078.0082196774. Delta is -7.14112389150614
c:\Users\alfredo.sampron\AppData\Local\miniconda3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no tran

>>>>>>>>>> BBAR 13 did not converge


Model is not converging.  Current: 17105.41849493449 is not greater than 17107.6124740924. Delta is -2.19397915790978
Model is not converging.  Current: 17043.366773032645 is not greater than 17061.395448137006. Delta is -18.028675104360445


In [ ]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]
    
    best_aic_nstate=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_nstate=results_dict_df[stock]["BIC"].astype(float).idxmin()
    print(f"For stock {stock}, best AIC: {best_aic_nstate} best BIC: {best_bic_nstate}")

    aic_best_model[stock]=hmm.GaussianHMM(n_components = best_aic_nstate, **param_dict).fit(insample_data)
    bic_best_model[stock]=hmm.GaussianHMM(n_components = best_bic_nstate, **param_dict).fit(insample_data)

Model is not converging.  Current: 18859.18589914981 is not greater than 18864.433488621307. Delta is -5.247589471498941
Model is not converging.  Current: 18150.45060648472 is not greater than 18151.012747677036. Delta is -0.5621411923166306


Model is not converging.  Current: 17767.738850783353 is not greater than 17768.03631132645. Delta is -0.2974605430972588


For stock ^MERV, best AIC: 4 best BIC: 3
For stock GGAL.BA, best AIC: 4 best BIC: 3


Model is not converging.  Current: 17746.98884698709 is not greater than 17749.887170814392. Delta is -2.898323827303102
Model is not converging.  Current: 17419.9513089545 is not greater than 17421.032240704913. Delta is -1.0809317504135834
Model is not converging.  Current: 17834.527199608863 is not greater than 17834.550895870856. Delta is -0.023696261992881773
Model is not converging.  Current: 17223.111725584055 is not greater than 17224.078189622516. Delta is -0.9664640384617087


For stock GGAL, best AIC: 4 best BIC: 3
For stock YPFD.BA, best AIC: 4 best BIC: 3
For stock YPF, best AIC: 4 best BIC: 3


...

Model is not converging.  Current: 16565.770178800347 is not greater than 16567.146166650873. Delta is -1.375987850526144
Model is not converging.  Current: 17444.741605353378 is not greater than 17449.044915339153. Delta is -4.303309985774831
Model is not converging.  Current: 16823.944155880148 is not greater than 16826.531123983852. Delta is -2.586968103703839
Model is not converging.  Current: 17365.257578606623 is not greater than 17370.854155502944. Delta is -5.596576896321494


For stock BMA.BA, best AIC: 4 best BIC: 3
For stock BMA, best AIC: 4 best BIC: 3
For stock BBAR.BA, best AIC: 4 best BIC: 3


Model is not converging.  Current: 17129.43925706523 is not greater than 17131.721490589865. Delta is -2.2822335246346483


For stock BBAR, best AIC: 4 best BIC: 3


Model is not converging.  Current: 17171.290878685777 is not greater than 17171.495405426213. Delta is -0.20452674043553998


# Generating out of sample data

In [ ]:
name=f'finaldf_test_{params["tablename"]}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df_test=pickle.load(handle)

In [1]:
model

NameError: name 'model' is not defined

In [ ]:
def generate_HMM_samples_residuals_2(model, insample_data, oos_data):
    """_summary_

    Args:
        model (_type_): _description_
        insample_data (_type_): _description_
        oos_data (_type_): _description_
    """
    # pseudocodigo
    # agarra el mejor modelo (esto con una cantidad optima de params ya esta)
    # fittear t-j con t-j-252d
    # Darle un año de datos hasta t-j para que me prediga la secuencia (probabilidad) de estados.
        # Le pido que me prediga las probabilidades de cada estado durante el periodo t-j, t-j-252: 
        # esto me da una matriz de (252 x n estados)
        # esto entiendo es https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.GaussianHMM.predict_proba
    # Tomo la ultima fila de la matriz
    # Multiplico esa por el vector de medias estimadas: este punto es mi forecast. 
        # esto es model.means (!)    
    
    split_date = oos_data.index[0]
    dates_to_forecast = len(oos_data.index)

    oos_data = pd.concat([insample_data, oos_data])
    del insample_data

    # vamos a implementar recursive window forecasting
    
    index = oos_data.index
    end_loc = np.where(index >= split_date)[0].min()
    # esto es un int del iloc
    # preciso usar ints de iloc porque el timedelta se me va a romper con el fin de semana
    rolling_window = 252

    forecasts = {}

    nstate=model
    model = hmm.GaussianHMM(n_components = nstate, **param_dict, verbose=False)

    for i in len(oos_data):
        

In [56]:
def generate_HMM_samples_residuals(model, insample_data, oos_data):
    """
    
    """
    # como el modelo es memoryless, sólo necesito 1 día de observación para saber en qué estado estoy
    # por lo tanto, en vez de complicarme con dos datasets, puedo agregarle el ultimo día de insample_data al ppio de oos_data
    # pseudocodigo
    oos_data=pd.concat([insample_data[-1:], oos_data])
    del insample_data

    samples=pd.DataFrame(columns=oos_data.columns)
    residuals=pd.DataFrame(columns=oos_data.columns)

    # for i=0
    for i in range(1,
                   len(oos_data.index)):
        prev_obs=oos_data[i-1:i]

        todays_obs = oos_data[i:i+1]
        todays_date = todays_obs.index

        state=model.decode(prev_obs)[1][-1]
        # decode()[0] is the log probability, decode()[1] is the sequence of states, [-1] is the last state
        # since we have added the last datum of insample_data to oos_data, then the 
            # TODO: revisar que tenga sentido decodear solo el ultimo día.
            # La alternativa es agregar diez días de insample al principio y usar un decode con diez dias, 
            # me quedo con el ultimo valor del array que maximiza la log-likelihood de la secuencia entera
            # pero como es memoryless, not sure if it makesense
        
        sample = model.sample(n_samples=1, random_state=random_state, currstate=state)[0]
        # sample()[0] is the array with observations of the sampled variables, sample()[1] is the value of the currstate
        sample = pd.DataFrame(data=sample, columns=oos_data.columns, index=todays_date)

        samples=pd.concat([samples, sample])   
        # sampling given state t-1
        # observar realización en t+i
        residual = todays_obs-sample
        
        residuals=pd.concat([residuals, residual])
    
    return samples, residuals

In [57]:
aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [60]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]
    oos_data=df_test[columns]

    samples, aic_best_residuals[stock] = generate_HMM_samples_residuals(
        aic_best_model[stock], 
        insample_data=insample_data, 
        oos_data=oos_data)

    samples, bic_best_residuals[stock] = generate_HMM_samples_residuals(
        bic_best_model[stock], 
        insample_data=insample_data, 
        oos_data=oos_data)

# Guardado de datos

In [62]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

In [63]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)

# Graficando

In [72]:
def plot_close_rets_vol(model, data, key, IC):
    prediction= model.predict(data)
    states=set(prediction)

    fig=plt.figure(figsize = (20, 20))
    plt.tight_layout()
    plt.title(f"{key} Log returns and intraday Vol\n{model.n_components} states / best by {IC}")

    for subplot, var in zip(range(1,3), data.columns):    
        plt.subplot(2,1,subplot)
        for i in set(prediction):
            state = (prediction == i)
            x = data.index[state]
            y = data[var].iloc[state]
            plt.plot(x, y, '.')
        plt.legend(states, fontsize=16)
        
        plt.grid(True)
        plt.xlabel("datetime", fontsize=16)
        plt.ylabel(var, fontsize=16)
            
    plt.savefig(os.path.join(resultsroute, "graphs", 
                             f"HMM", 
                             f"{key}_model_{IC}.png"))

In [ ]:
for dictionary, IC in zip([aic_best_model, bic_best_model], ["AIC", "BIC"]):
    for key, model in dictionary.items():
        columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
        insample_data = df[columns]
        oos_data=df_test[columns]
        train_end=insample_data.index.max()
        data=pd.concat([insample_data, oos_data])

        plot_close_rets_vol(model, data, key, IC)

## HMM Selection

Selecting the Number of States in Hidden Markov Models: Pragmatic Solutions Illustrated Using Animal Movement
https://sci-hub.st/10.1007/s13253-017-0283-8